# Clase 2: Librería Chainladder

<a href="https://colab.research.google.com/github/andesanalytics/python/blob/main/docs/modulo_4/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción a Chainladder

+ La biblioteca `chainladder` de Python es una herramienta utilizada en el análisis actuarial,
principalmente en el ámbito de seguros.

+ Su principal función es ayudar en la estimación de reservas de siniestros ocurridos y no reportados.

+ La librería posee multiples técnicas de proyección, que se basan en el análisis de triángulos de desarrollo de pérdidas, que
permiten proyectar los siniestros futuros basados en los datos históricos


## Instalación y Configuración

Para instalar la biblioteca `chainladder`, ejecuta el siguiente comando en tu entorno Python:


In [ ]:
pip install chainladder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00


Una vez instalada, puedes importarla y verificar su versión para asegurarte de que está correctamente instalada:

In [ ]:
import chainladder as cl
print(cl.__version__)

0.8.18


## Manejo de Datos: Propiedades Básicas
+ El paquete chainladder tiene su propia estructura de datos `:class:Triangle` que se comporta de manera muy similar a un `DataFrame` de ``pandas``
+ Por qué no ``pandas``? Principalmente porque podemos manejar multiples triángulos al mismo tiempo, junto con la eficiencia en memoria
+ La estructura `Triangle` es una estructura de datos en 4D con ejes etiquetados. Estos ejes son ``index``, ``columns``, ``origin``, ``development``


``index`` (axis 0): Es la agrupación mínima a la cual se requiera ver los datos (por ejemplo por producto, tipo de negocio, ramo fecu). Tal como en ``pandas.multiIndex``, se pueden tener más de una columa dentro de `index`

``columns`` (axis 1): Son el o los valores numéricos que deseas agrupar en triángulo. Si bien el valor obvio a acumular es el de siniestros pagados, también puedes tener otros triángulos como el número de siniestros, o los siniestros pendientes por ejemplo.

``origin`` (axis 2): Es el eje vertical dentro de un triángulo tradicional. Corresponde al mes, trimestre, semestre o año en el cual ocurrió el siniestro. La periodicidad se puede definir por el usuario.

``development`` (axis 3): Es el eje horizontal del triángulo. Representa el periodo en que el siniestro fue reportado. Al igual que el eje `origin` se puede tomar la periodicidad mensual, trimestral, semestral, anual.

### ¿Cómo se trabaja con una clase ``Triangle``?
+ Independiente de poseer 4 dimensiones, esta clase permite que se interactúe con él de la misma manera que en `pandas`. Puedes usar los ejes `index` y `columns` de la misma manera que en un `DataFrame`, donde cada elemento será un triángulo tradicional. La siguiente imagen muestra como sería la estructura de la clase.

<figure>
<center>
<img src="https://drive.google.com/uc?id=17yiVi-2aFOBuAWkuElkgOFG2iJ7gUuyZ" />
<figcaption>Estructura Clase Triangle</figcaption></center>
</figure>

### ¿Cómo se construye una clase ``Triangle``?

+ Una clase `Triangle` se crea a partir de un `DataFrame` de `pandas`, el cuál debe poseer como mínimo las siguientes columnas.
    * Dos columnas de tipo fecha que representen los ejes ``origin`` y ``development``.
    * Una columna numérica que represente el eje ``columns``

<figure>
<center>
<img src="https://drive.google.com/uc?id=1OLd4ccAVEWor9yssbB0X5dsAcc2VMokl" />
<figcaption>Formato dataframe a cargar</figcaption></center>
</figure>




In [1]:
import pandas as pd
raa_df = pd.read_csv("https://raw.githubusercontent.com/casact/chainladder-python/master/chainladder/utils/data/raa.csv")
raa_df.head(10)

,development,origin,values
0,1981,1981,5012.0
1,1982,1982,106.0
2,1983,1983,3410.0
3,1984,1984,5655.0
4,1985,1985,1092.0
5,1986,1986,1513.0
6,1987,1987,557.0
7,1988,1988,1351.0
8,1989,1989,3133.0
9,1990,1990,2063.0


Vemos que dataframe cargado posee las columnas mínimas requeridas.
La forma para obtener un objeto de tipo `Triangle` se obtiene con el método `Triangle`

In [ ]:
raa = cl.Triangle(raa_df,origin="origin",development="development",columns="values",cumulative=True)
print(raa.shape)
raa

+ Como se puede observar, la información fue agrupada como triángulo.
+ Las dimensiones del triángulo indican 4 dimensiones, que corresponden a los ejes anteriormente comentados
+ La visualización del triángulo cambiará si nuestra data contiene más de un valor en el eje ``columns`` o más de un valor en el eje `index`

### ``Triangle`` en 4D
Ahora revisemos el siguiente triangulo cargado de los datos de muestra de la librería

In [ ]:
triangle = cl.load_sample('CLRD')
print(triangle.shape)
triangle

Vemos que la forma de mostrar la data cambia, debido a que el eje ``index`` posee 775 diferentes niveles, mientras que el eje `columns` posee 6 distintas columnas para formar triángulos.

### Parametro `cumulative`
+ El parámetro ``cumulative`` indica si los datos que entregamos en el eje `columns` corresponden a datos acumulativos a la fecha o son los datos específicos de la fecha.
+ Es completamente opcional. Si no se especifica, el Triángulo inferirá su estado ``cumulative/incremental`` en el punto en el que llame a los métodos ``cum_to_incr`` o ``incr_to_cum`` (que sirven para pasar de un estado a otro)
+ El método `is_cumulative` nos permite saber si un triangulo tiene su parametro `cumulative` en `True` o `False`

In [ ]:
raa.cum_to_inc()

### Valoración vs Desarrollo
Si bien la mayoría de los estimadores que utilizan triángulos esperan que el período de desarrollo se exprese como una edad sobre el periodo de origen (diagonal de abajo hacia arriba), es posible transformar un triángulo en un triángulo de valoración, donde los períodos de desarrollo se convierten en períodos de valoración (es decir con la diagonal de arriba hacia abajo) con el método `dev_to_val`

Los triangulos poseen también la propiedad `is_val_tri` que indica si el triangulo está en modo valoración (diagonal de arriba hacia abajo)

In [ ]:
raa.dev_to_val()
raa.is_val_tri

### Granularidad del Triangulo
Si el triángulo posee en sus ejes `origin` o `development` periodicidades más frecuentes a una anual (mensual, trimestral), se puede facilmente pasar a una granularidad mayor con el metodo `grain`. Este método reconoce las siguientes granularidades.
+ Anual: Yearly (Y)
+ Trimestral: Quarterly (Q)
+ Mensual: Monthly (M)

In [ ]:
df_quarterly=cl.load_sample('quarterly')
df_quarterly
df_quarterly.grain('OYDY')

### Link Ratios
Los factores de desarrollo por cada periodo origen-desarrollo (también conocidos en inglés como *link ratios*) pueden ser visualizados con la propiedad ``link_ratio``. Los triángulos además tienen un método llamado ``heatmap`` que puedes llamar opcionalmente para darle formatos a las salidas del triángulo. Este metodo requiere de IPython/Jupyter notebook para ejecutarse.

In [ ]:
triangle_abc = cl.load_sample('abc')
triangle_abc
triangle_abc.link_ratio
triangle_abc.link_ratio.heatmap()

## Sintaxis Tipo Pandas
### Filtros y Slicing
Podemos filtrar y seleccionar triángulos de menor tamaño al igual que en pandas. Funciones como `iloc` o `loc` aplican de la misma manera.

**Recordar que bajo la lógica de esta librería el eje `index` vendría siendo el eje de filas y el eje `columns` el de columnas en pandas.**

In [ ]:
clrd = cl.load_sample('clrd')
clrd.iloc[0,1]
clrd[clrd['LOB']=='othliab']
clrd['EarnedPremDIR']

### Virtual Columns
Hay casos en los que queremos posponer los cálculos, podemos crear columnas "virtuales" que pospongan los cálculos cuando sea necesario. Estas columnas se pueden crear envolviendo una columna normal en una función. Las expresiones lambda funcionan como una representación ordenada de columnas virtuales.

In [ ]:
clrd = cl.load_sample('clrd')
clrd['PaidLossRatio'] = clrd['CumPaidLoss'] / clrd['EarnedPremDIR']
clrd.sum()['PaidLossRatio']
clrd['PaidLossRatio'] = lambda clrd : clrd['CumPaidLoss'] / clrd['EarnedPremDIR']
clrd.sum()['PaidLossRatio']

### Agregaciones
Al igual que los pandas, puedes agregar múltiples triángulos dentro de un Triángulo usando ``sum()`` que opcionalmente puede combinarse con ``groupby()``.

In [ ]:
clrd = cl.load_sample('clrd')
clrd.sum()
clrd.groupby('LOB').sum()

* Por defecto, la agregación se aplicará al primer eje con una longitud mayor que 1.
* Alternativamente, puede especificar el eje usando el argumento ``axis`` del método agregado.

### Convertir a ``DataFrame``
Cuando un triángulo se presenta con un único nivel del eje ``index`` y una sola columna (eje `column`), se convierte en un objeto 2D (En un triángulo como vimos al comienzo). Como tal, su formato de visualización cambia a uno similar a un marco de datos. Estos triángulos 2D se pueden convertir fácilmente en un marco de datos de pandas utilizando el método ``to_frame``. A partir de este punto, los resultados se pueden operar directamente en pandas.

La funcionalidad ``to_frame`` funciona cuando un Triángulo se divide en dos ejes cualesquiera y no se limita solo a `index` y `column`

In [ ]:
clrd = cl.load_sample('clrd')
clrd
clrd[clrd['LOB']=='ppauto']['CumPaidLoss'].sum().to_frame()
clrd['CumPaidLoss'].groupby('LOB').sum().latest_diagonal.to_frame()

## Desarrollo de Triangulos
Para realizar esto utilizamos `Development`, que permite la selección de patrones de desarrollo de pérdidas. Muchas de las técnicas típicas de promediación están disponibles en esta clase: simple, de volumen y de regresión a través del origen. Además, Desarrollo incluye patrones para permitir una exclusión precisa de las relaciones de enlace del cálculo de LDF.

**Alternativamente, puedes proporcionar una lista para parametrizar cada período de desarrollo por separado. Debes tener en consideración que la lista debe tener la misma longitud que el eje de desarrollo de sus triángulos ``link_ratio``**

In [ ]:
raa = cl.load_sample('raa')
cl.Development(average='simple')
len(raa.link_ratio.development)
cl.Development(average=['volume']+['simple']*8)

### Omitiendo Link Ratios
Hay varios argumentos para eliminar celdas individuales del triángulo, así como para excluir períodos de valoración completos o máximos y mínimos. Se permite cualquier combinación de los argumentos de "drop".

**Cuando se utiliza ``drop``, se debe hacer referencia a la edad más temprana de ``link_ratio``. Por ejemplo, utilice 12 para eliminar la proporción 12-24.**

In [ ]:
cl.Development(drop_high=[True]*5+[False]*4, drop_low=[True]*5+[False]*4).fit(raa)
cl.Development(drop_valuation='1985').fit(raa)
cl.Development(drop=[('1985', 12), ('1987', 24)]).fit(raa)
cl.Development(drop=('1985', 12), drop_valuation='1988').fit(raa)

### Transformación de Triangulos
Al transformar un Triángulo, recibirás una copia del triángulo original junto con las propiedades ajustadas del estimador `Development`.

Mientras que el Triángulo original contiene todas los link ratios, la versión transformada reconoce cualquier omisión que especifique.

In [ ]:
triangle = cl.load_sample('raa')
dev = cl.Development(drop=('1982', 12), drop_valuation='1988')
transformed_triangle = dev.fit_transform(triangle)
transformed_triangle.ldf_

In [ ]:
transformed_triangle.link_ratio.heatmap()

Al desacoplar los métodos de ajuste y transformación, podemos aplicar nuestro estimador de desarrollo a nuevos datos. Este es un patrón común en ``scikit-learn``. En este ejemplo generamos patrones de desarrollo a nivel industrial y aplicamos esos patrones a empresas individuales.

In [ ]:
clrd = cl.load_sample('clrd')
clrd = clrd[clrd['LOB']=='wkcomp']['CumPaidLoss']
# Summarize Triangle to industry level to estimate patterns
dev = cl.Development().fit(clrd.sum())
# Apply Industry patterns to individual companies
dev.transform(clrd)

El estimador ``DevelopmentConstant`` simplemente le permite codificar patrones de desarrollo en un Estimador de desarrollo. Un ejemplo común sería incluir un conjunto de patrones de desarrollo de la industria en su flujo de trabajo que no se estimen directamente a partir de ninguno de sus propios datos.

In [ ]:
triangle = cl.load_sample('ukmotor')
patterns={12: 2, 24: 1.25, 36: 1.1, 48: 1.08, 60: 1.05, 72: 1.02}
cl.DevelopmentConstant(patterns=patterns, style='ldf').fit(triangle).ldf_

## Estimadores de cola
LaS pérdidas no observadas más allá del borde de un Triángulo puedeN ser sustancialES y es una parte necesaria del análisis de un actuario. Esto es particularmente cierto para las líneas de cola larga, más comunes en seguros comerciales o niveles excesivos de pérdidas (seguros de reembolsos de gastos médicos de alto costo).

Con todas las estimaciones de cola, estamos extrapolando más allá de los datos conocidos, lo que conlleva sus propios desafíos. Tiende a ser más difícil validar los supuestos cuando se realiza una estimación de cola. Además, muchas de las técnicas conllevan un alto grado de sensibilidad a los supuestos que se elegirán al realizar el análisis.

Sin embargo, es una parte necesaria del conjunto de herramientas del actuario para estimar los pasivos de reserva.

### Conceptos básicos y puntos en común
+ Al igual que con la familia de estimadores `Development`, el módulo ``Tail`` proporciona una variedad de transformadores de cola que permiten la extrapolación de patrones de desarrollo más allá del final del triángulo.
+ Al ser transformadores, podemos esperar que cada estimador tenga un método ``fit`` y otro método `transform `.
+ El método `transform` nos devuelve nuestro Triángulo con parámetros estimados adicionales para incorporar nuestra revisión de cola en nuestras estimaciones de IBNR.
+ Cada estimador ``tail`` produjo un atributo ``tail_`` que representa la estimación puntual de la cola del Triángulo.

Veamos el primer ejemplo de una estimación con un valor asignado arbitrariamente.

In [ ]:
triangle = cl.load_sample('genins')
tail = cl.TailConstant(1.10).fit_transform(triangle)
tail.tail_

**Al ajustar un estimador ``tail`` sin especificar primero un estimador ``Development``, chainladder asume un desarrollo de triangulos con el parametro `volume-weighted`. Para anular esto, primero debes declarar transformar tu triángulo con un transformador de desarrollo.**

### Estimador `TailCurve`

In [ ]:
triangle = cl.load_sample('quarterly')['paid']
tail = cl.TailCurve().fit(triangle)
# Slice just the tail entries
tail.cdf_[~tail.ldf_.development.isin(triangle.link_ratio.development)]

### Comienzo de la Estimación
Por defecto, los estimadores de cola se asignan a la edad de desarrollo más antigua del Triángulo. En la práctica, los últimos factores de desarrollo conocidos de un Triángulo pueden ser poco confiables y se prefiere unir la cola antes y usarla como mecanismo de suavizado. Todos los estimadores de cola tienen un parámetro ``attachment_age``  que le permite seleccionar la edad de desarrollo a la que se unirá la cola.

In [ ]:
triangle = cl.load_sample('genins')
unsmoothed = cl.TailCurve().fit(triangle).ldf_
smoothed = cl.TailCurve(attachment_age=24).fit(triangle).ldf_
pd.concat((
    unsmoothed.T.iloc[:, 0].rename('Unsmoothed'),
    smoothed.T.iloc[:, 0].rename('Age 24+ Smoothed')),
    axis=1).plot(marker='o', title='Selected Link Ratio', ylabel='LDF');

### Periodo de Proyección
Independientemente de dónde se coloque una cola, habrá patrones incrementales hasta al menos un año después del final del Triángulo. Surgen casos en los que se desea modelar  en un horizonte temporal más largo. Para estos casos, es posible modificar el período de proyección (en meses) de todos los estimadores de cola especificando la cantidad de meses que desea extender sus patrones (dentro del parametro `projection_period`).

In [ ]:
# Extend tail run-off 4 years past end of Triangle.
tail = cl.TailCurve(projection_period=12*4).fit(triangle)
tail.ldf_[~tail.ldf_.development.isin(triangle.link_ratio.development)]

### Tasa de Caida
Se puede controlar la tasa de caída de su cola. También está disponible un parámetro exponencial `decay`

In [ ]:
tail = cl.TailConstant(tail=1.05, decay=0.95).fit_transform(triangle)
tail.ldf_
tail.tail_

## Modelos de OYNR - Definiciones en Comun
* Los Estimadores IBNR son la etapa final en el análisis de las estimaciones de reservas en el paquete Chainladder.
* Estos estimadores tienen un método de predicción en lugar de un método de transformación.
* Todos los métodos de reserva OYNR determinan el costo final de las reclamaciones de seguros (**ultimate claims** en inglés). Estos últimos se capturan en el atributo ``ultimate_`` del estimador.
* La diferencia entre los atributos `ultimate_` y `latest_diagonal` de un triangulo son guardados en el atributo `ibnr_` del estimador.

In [ ]:
cl.Chainladder().fit(cl.load_sample('raa')).ultimate_

In [ ]:
triangle = cl.load_sample('quarterly')
model = cl.Chainladder().fit(triangle)
case_reserves = (triangle['incurred']-triangle['paid']).latest_diagonal
true_ibnr = model.ibnr_['paid'] - case_reserves
true_ibnr.sum()

### Triangulos Completados
* Los atributos `full_triangle_` y `full_expectation_` dan una vista del triangulo completado
* `full_expectation_` Está basado en los valores que entrega el atributo `ultimate_` junto con los factores de desarrollo
* `full_triangle_` es una mezcla del triangulo real más lo proyectado de acuerdo al método de completado

In [ ]:
model = cl.Chainladder().fit(cl.load_sample('ukmotor'))
residuals = model.full_expectation_ - model.full_triangle_
residuals[residuals.valuation<=model.X_.valuation_date]

Otro análisis típico es predecir el OYNR para períodos futuros.

In [ ]:
expected_3y_run_off = model.full_triangle_.dev_to_val().cum_to_incr().loc[..., '2014':'2016']
expected_3y_run_off

## Modelos de OYNR - Metodologias
### `Chainladder`
* Metodo mas basico de estimación de OYNR
* La característica distintiva del método ``:class:Chainladder`` es que las reclamaciones finales para cada año de accidente se generan a partir de valores registrados, suponiendo que el desarrollo de las reclamaciones futuras es similar al desarrollo de años anteriores.
* Algunos supuestos subyacentes del modelo son:
    * Los siniestros registrados hasta la fecha continuarán desarrollándose de manera similar en el futuro
    * Para un año de accidentes reciente, los siniestros observados hasta ahora le dicen algo sobre los siniestros aún por observar. Esto contrasta con los supuestos subyacentes a la técnica de reclamaciones esperadas (expected loss ratio).
    * Procesamiento consistente de reclamos
    * Combinación estable de tipos de reclamos
    * Límites de póliza estables
### `MackChainladder`
* Puede considerarse como una forma especial de regresión lineal ponderada a través del origen para cada período de desarrollo.
* Dado lo anterior, las estadísticas sobre la variabilidad de los datos y las estimaciones de los parámetros permiten estimar los errores de predicción
* Es el método más básico de los métodos estocásticos.
* Solo es compatible con los estimadores ``Development`` y ``TailCurve``.

### `BornhuetterFerguson`
* Es esencialmente una combinación de las técnicas de desarrollo y de reclamaciones esperadas
* En la técnica de desarrollo de triangulos
    * Multiplicamos las reclamaciones reales por un factor de desarrollo de reclamaciones acumulativo
    * **Esta técnica puede conducir a proyecciones erráticas y poco confiables cuando el factor de desarrollo acumulativo es grande porque una variación relativamente pequeña en las reclamaciones reportadas o la presentación de una reclamación inusualmente grande podría resultar en una variación muy grande en las reclamaciones finales proyectadas**
* En la técnica de reclamaciones esperadas
    * La estimación de reclamaciones impagas es igual a la diferencia entre una estimación predeterminada de reclamaciones esperadas y los pagos reales
    * **Esto tiene la ventaja de la estabilidad, pero ignora por completo los resultados reales informados**

* La técnica de Bornhuetter-Ferguson combina las dos técnicas al dividir los reclamos finales en dos componentes: reclamos reales reportados (o pagados) y reclamos esperados no reportados (o no pagados). A medida que la experiencia madura, se le da más peso a las reclamaciones reales y las reclamaciones esperadas se vuelven gradualmente menos importantes.
* Es uno de los tantos estimadores que soporta una medida de exposicion. Para esto, debemos agregarla durante el método `fit` a traves del argumento `sample_weight`, entregando una triángulo que represente la exposición.
* Debemos tener una estimacion *a priori* de la siniestralidad esperada podemos usar el argumento `a priori` para establecerlo dentro del estimador.

In [ ]:
cl_ult = cl.Chainladder().fit(raa).ultimate_ # Chainladder Ultimate
apriori = cl_ult*0+(cl_ult.sum()/10) # Mean Chainladder Ultimate
cl.BornhuetterFerguson(apriori=1).fit(raa, sample_weight=apriori).ultimate_

### ``Benktander``
* Es un promedio ponderado por factoeres de credibilidad de las técnica ``BornhuetterFerguson`` y `Chainladder`
* La ventaja citada por los autores es que este método resultará más receptivo que la técnica de Bornhuetter-Ferguson y más estable que la técnica de Chainladder.

$Ultimate = Apriori\times (1-\frac{1}{CDF})^{n} + Latest\times \sum_{k=0}^{n-1}(1-\frac{1}{CDF})^{k}$
* Donde
    * n=0 corresponde al método de pérdida esperada.
    * n=1 es el método BornhuetterFerguson.
    * n>>1 converge al metodo Chainladder.

### ``CapeCod``
* También conocido como método de Stanard-Buhlmann, es similar a la técnica de Bornhuetter-Ferguson
* La principal diferencia entre los dos métodos es la derivación del índice de siniestralidad esperado.
* Se obtiene del propio triángulo en lugar de una selección independiente y a menudo basada en criterios como en la técnica de Bornhuetter-Ferguson


## Anexo - Resumen de Estructuras de Datos y Estructura de `Chainladder`
### ¿Qué es una clase?
* Las clases proporcionan un medio para agrupar datos y funcionalidades.
* La creación de una nueva clase crea un nuevo tipo de objeto, lo que permite crear nuevas instancias de ese tipo.
* En otras palabras, una clase es una plantilla de código para crear objetos.

### Atributos y Métodos dentro de una Clase:
Una clase por sí sola no sirve de nada a menos que haya alguna funcionalidad asociada a ella. Las funcionalidades se definen estableciendo atributos, que actúan como contenedores para datos y funciones relacionadas con esos atributos. Esas funciones se llaman métodos.

### ``scikit-learn`` y `Chainladder`
La librería ``scikit-learn`` es una interfaz de modelado común que se utiliza para construir y adaptar una variedad incontable de algoritmos de aprendizaje automático. La interfaz común permite un intercambio muy rápido entre modelos con cambios mínimos de código. El paquete ``Chainladder`` ha adoptado la interfaz para promover un enfoque estandarizado para adaptar modelos de reserva. Esto es, usa su misma estructura de objetos y clases.

### Estimadores
* Es un tipo de objeto.
* Todos los objetos del tipo estimador se pueden configurar opcionalmente con parámetros para especificar de forma única el modelo que se está construyendo. Esto se hace antes de enviar cualquier dato a través del modelo.
* Todos los estimadores en `Chainladder` poseen una de las siguientes caracteristicas:
    * Poseen el método `transform`, que transforma el Triángulo de entrada y le otorga nueos atributos
    * Predicen un resultado con el método `predict`.

```python
estimator = Estimator(param1=1, param2=2)
```

Todos los estimadores en la librería ``Chainladder``poseen un método llamado `fit` que toma una `Triangle` como input.

```python
estimator.fit(X=data)

```

Todos los estimadores incluyen una opción ``sample_weight`` en el método ``fit`` para especificar una base de exposición. Si una base de exposición no es aplicable, entonces se ignora este argumento.

```python
estimator.fit(X=data, sample_weight=weight)
```


### Transformers
En ``scikit-learn``, un transformador se refiere a un objeto que transforma un conjunto de datos en uno nuevo para preparar el conjunto de datos para el modelado predictivo.

Exceptuando los modelos IBNR finales, los estimadores de ``chainladder`` son transformadores. Es decir, devuelven el objeto `Triangle` con propiedades adicionales.

```python
# Fitting and Transforming
estimator.fit(data)
transformed_data = estimator.transform(data)
# One line equivalent
transformed_data = estimator.fit_transform(data)
```

### Predictors
Todos los predictores incluyen un método de predicción.

````python
prediction = estimator.predict(new_data)
````

Los predictores están destinados a crear nuevas predicciones. No es raro ajustar un modelo sobre una data más agregada (digamos a nivel nacional) y predecir en un triángulo más granular (estatal o provincial).



